In [43]:
import pandas as pd
import numpy as np
from pprint import pprint


In [44]:
df = pd.read_csv('steam-200k.csv')
user_dict = dict()
game_dict = dict()
print(df.columns)

print(df.tail(2))

Index(['user_id', 'name', 'status', 'hours'], dtype='object')
          user_id  name    status  hours
199998  128470551  RUSH  purchase    1.0
199999  128470551  RUSH      play    1.4


In [69]:
df = df[(df['hours']>=2) & (df['status']=='play')]
df_filtered = df[df.groupby('name').user_id.transform(len) >= 20]
df_filtered['name'].nunique()
df_filtered.head(20)

df_filtered['user_id'] = df_filtered['user_id'].astype(str)
averages = df_filtered.groupby(['name'],as_index=False).hours.mean()
# averages.loc[averages['name']=="The Witcher 3 Wild Hunt"]
# print(float(averages[averages["name"]=="The Witcher 3 Wild Hunt"]['hours']))
# df_filtered['rating'] = df_filtered.apply(get_rating,axis=1)
# df_filtered.head(10)

# conditions = [
#     df_filtered['hours']>=0.8*averages[averages['name'] == df_filtered['name']['hours']],
#     df_filtered['hours']>=0.6*averages[averages['name'] == df_filtered['name']['hours']],
#     df_filtered['hours']>=0.4*averages[averages['name'] == df_filtered['name']['hours']],
#     df_filtered['hours']>=0.2*averages[averages['name'] == df_filtered['name']['hours']],
#     df_filtered['hours']>=0,
# ]
# values = [5,4,3,2,1]
# float(averages[averages['name']==df_filtered['name']]['hours'])
# df_filtered['rating'] = np.select(conditions,values)
df_filtered['avg_hours'] = float(averages.iat[averages.iloc[averages['name']==df_filtered['name']]]['hours'])
# df_filtered.head(10)


ValueError: Can only compare identically-labeled Series objects

In [8]:
# # avg_game_time = dict()
# # for user,value in user_dictionary.items():
# #     for game,time in value.items():
# #         avg_game_time[game] = 
# user_dictionary = dict(user_dict) #copy over user dictionary
# # pprint(user_dictionary)
# avg_game_time = dict(mod_dict)
# # print(avg_game_time['The Witcher 3 Wild Hunt'])
# # print(len(avg_game_time))
# for game,value in avg_game_time.items():
#     avg_game_time[game] = value[1]/value[0]
# print(user_dictionary[292652586])
# # print(avg_game_time['Her Story'])
# deletable_game = []
# for user,value in user_dictionary.items():
#     for game,time in value.items():
#         # print(game)
#         try:
#             if time>=avg_game_time[game]*0.8:
#                 user_dictionary[user][game] = 5
#             elif time>=avg_game_time[game]*0.6:
#                 user_dictionary[user][game] = 4
#             elif time>=avg_game_time[game]*0.4:
#                 user_dictionary[user][game] = 3
#             elif time>=avg_game_time[game]*0.2:
#                 user_dictionary[user][game] = 2
#             elif time>=0:
#                 user_dictionary[user][game] = 1
#             else:
#                 print("Reached here tf\n")
#         except:
#             deletable_game.append(game)
#             continue
# for user in user_dictionary.keys():
#     for game in deletable_game:
#         try:
#             del user_dictionary[game]
#         except:
#             continue
# pprint(user_dictionary)
        